In [44]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [45]:
#reading the file
df=pd.read_excel("Formatted_Data.xlsx")
df.head()

,Dilation_Average,Erosion_Average,C_Kurtosis,C_ Max,C_Skewness,C_Variance,D_Kurtosis,D_ Max,D_Skewness,D_Variance,...,E_Skewness,E_Variance,O_Kurtosis,O_ Max,O_Skewness,O_Variance,Nc_D,Nc_E,Nmsw_D,Class
0,1.003223,-1.003223,1.525407,0.472831,0.000100,0.100024,1.542018,1.472831,-0.008037,0.100279,...,0.008037,0.100279,1.525407,0.471576,-0.000100,0.100024,1.257324,-0.519413,101.482714,Very_Light
1,0.993225,-1.013220,1.525406,0.462831,0.000054,0.100024,1.541202,1.462831,-0.007831,0.100274,...,0.008248,0.100284,1.525406,0.461576,-0.000146,0.100024,1.259912,-0.494549,101.587092,Very_Light
2,0.998224,-1.008222,1.525407,0.467831,0.000077,0.100024,1.541607,1.467831,-0.007933,0.100277,...,0.008142,0.100281,1.525407,0.466576,-0.000123,0.100024,1.258611,-0.506919,101.534655,Very_Light
3,1.000224,-1.006222,1.525407,0.469831,0.000086,0.100024,1.541771,1.469831,-0.007975,0.100277,...,0.008100,0.100280,1.525407,0.468576,-0.000114,0.100024,1.258095,-0.511902,101.513820,Very_Light
4,1.005222,-1.001223,1.525407,0.474831,0.000109,0.100024,1.542185,1.474831,-0.008079,0.100280,...,0.007995,0.100278,1.525407,0.473576,-0.000091,0.100024,1.256813,-0.524445,101.462075,Very_Light


In [46]:
#all classifier definition
#random forest function

def _randomForestClassify(x_train,x_test,y_train,y_test):
    classifier=RandomForestClassifier(n_estimators=100,random_state=0,n_jobs=1)
    classifier.fit(x_train,y_train)
    _predictedY=classifier.predict(x_test)
    print("Accuarcy of the Random Forest Classifier is: ")
    print(accuracy_score(y_test,_predictedY))

In [47]:
#feature axis and class axis
features=df.drop("Class",axis=1)
output=df[['Class']]
print(features.shape," ",output.shape)

#train test split
x_train,x_test,y_train,y_test=train_test_split(features,output,test_size=0.2,random_state=4)
print(x_train.shape," ",x_test.shape)

#constant value removal
from sklearn.feature_selection import VarianceThreshold


_filterToRemoveConstant=VarianceThreshold(threshold=0)
_filterToRemoveConstant.fit(x_train)
print(_filterToRemoveConstant.get_support().sum())

constant_features=[not temp for temp in _filterToRemoveConstant.get_support()]
constant_features=features.columns[constant_features]

#removing those features
for feature_index in constant_features:
    x_train.drop(feature_index,axis=1)
    x_test.drop(feature_index,axis=1)
print(x_train.shape," ",x_test.shape)

#remove low variance features
_lowVarianceFeatures=VarianceThreshold(threshold=0.1)
_lowVarianceFeatures.fit(x_train)
print(_lowVarianceFeatures.get_support().sum())#return features that are of high variance

#printing low variance features
low_variance_features=[not temp for temp in _lowVarianceFeatures.get_support()]
print("Low Variance Features", features.columns[low_variance_features])

#printing high variance features
high_variance_features=[temp for temp in _lowVarianceFeatures.get_support()]
print("High Variance Features ",features.columns[high_variance_features])

#remoing the low variance features
low_variance_features=np.asarray(features.columns[low_variance_features])
for feature_index in low_variance_features:
    x_train.drop(feature_index,axis=1,inplace=True)
    x_test.drop(feature_index,axis=1,inplace=True)

print(x_train.shape," ",x_test.shape)

#remove duplicate features
_transposeTrain=x_train.T
_transposeTest=x_test.T

duplicate_features=_transposeTrain.duplicated() #returns true if duplicated
duplicate_features=np.asarray(x_train.columns[duplicate_features])
for feature_index in duplicate_features:
    x_train.drop(feature_index,axis=1)
    x_test.drop(feature_index,axis=1)
print(x_train.shape," ",x_test.shape)

_randomForestClassify(x_train,x_test,y_train,y_test)

(25, 21)   (25, 1)
(20, 21)   (5, 21)
21
(20, 21)   (5, 21)
11
Low Variance Features Index(['Dilation_Average', 'Erosion_Average', 'C_Kurtosis', 'C_Skewness',
       'D_Kurtosis', 'D_Skewness', 'E_Kurtosis', 'E_Skewness', 'O_Kurtosis',
       'O_Skewness'],
      dtype='object')
High Variance Features  Index(['C_ Max', 'C_Variance', 'D_ Max', 'D_Variance', 'E_ Max', 'E_Variance',
       'O_ Max', 'O_Variance', 'Nc_D', 'Nc_E', 'Nmsw_D'],
      dtype='object')
(20, 11)   (5, 11)
(20, 11)   (5, 11)


C:\Users\Abhinaba\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Abhinaba\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Accuarcy of the Random Forest Classifier is: 
1.0
